<a href="https://colab.research.google.com/github/Karan1928/Python-for-DS/blob/main/DecisionTree_FraudCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot

In [6]:
fraudcheck=pd.read_csv('Fraud_check.csv')
fraudcheck

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [8]:
fraudcheck["income"]="<=30000"
fraudcheck.loc[fraudcheck["Taxable.Income"]>=30000,"income"]="Good"
fraudcheck.loc[fraudcheck["Taxable.Income"]<=30000,"income"]="Risky"

In [9]:
#Droping the Taxable income variable
fraudcheck.drop(["Taxable.Income"],axis=1,inplace=True)

In [10]:
fraudcheck.rename(columns={"Undergrad":"undergrad","Marital.Status":"marital","City.Population":"population","Work.Experience":"experience","Urban":"urban"},inplace=True)

In [11]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

In [16]:
for column_name in fraudcheck.columns:
    if fraudcheck[column_name].dtype == object:
        fraudcheck[column_name] = le.fit_transform(fraudcheck[column_name])
    else:
        pass

In [18]:
#Splitting the data into featuers and labels
features = fraudcheck.iloc[:,0:5]
labels = fraudcheck.iloc[:,5]

In [19]:
# Collecting the column names
colnames = list(fraudcheck.columns)
predictors = colnames[0:5]
target = colnames[5]

In [20]:
#Splitting the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(features,labels,test_size = 0.2,stratify = labels)

In [21]:
#Model building
from sklearn.ensemble import RandomForestClassifier as RF
model = RF(n_jobs = 3,n_estimators = 15, oob_score = True, criterion = "entropy")
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=3,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [24]:
model.estimators_
model.classes_
model.n_features_
model.n_classes_

2

In [25]:
model.n_outputs_

1

In [26]:
model.oob_score_

0.7083333333333334

In [27]:
#Predictions on train data
prediction = model.predict(x_train)

In [28]:
#Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_train,prediction)

In [30]:
print(accuracy)

0.9833333333333333


In [31]:
np.mean(prediction == y_train)

0.9833333333333333

In [32]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_train,prediction)

In [33]:
#Prediction on test data
pred_test = model.predict(x_test)

In [34]:
#Accuracy
acc_test =accuracy_score(y_test,pred_test)

In [38]:
print(acc_test)

0.725


In [40]:
from sklearn.tree import export_graphviz
import pydotplus
from sklearn.externals.six import StringIO

In [41]:
tree = model.estimators_[5]

In [42]:
dot_data = StringIO()
export_graphviz(tree,out_file = dot_data, filled = True,rounded = True, feature_names = predictors ,class_names = target,impurity =False)

In [43]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [44]:
# Creating pdf and png file the selected decision tree
graph.write_pdf('fraudrf.pdf')
graph.write_png('fraudrf.png')

True